# Merge original data with new sentiment and topic classification

In [13]:
import pandas as pd

In [14]:
data = pd.read_csv("./data/eastmoney_bert.csv", usecols=['item_title', 'item_author_cate', 'article_author', 'article_source_cate',
       'year', 'month',
       'eastmoney_robo_journalism', 'media_robo_journalism', 'SMA_robo_journalism', 
       'stock_code',
       'viral'])

C:\Users\ZIQING002\AppData\Local\Temp\ipykernel_31388\3179780348.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("./data/eastmoney_bert.csv", usecols=['item_title', 'item_author_cate', 'article_author', 'article_source_cate',


In [15]:
sentiment_data = pd.read_csv("./lda/sentiment_score.csv", usecols=['sentiment_label'])
sentiment_data.columns

Index(['sentiment_label'], dtype='object')

In [16]:
data['sentiment'] = sentiment_data['sentiment_label']

In [17]:
topic_data = pd.read_csv("./lda/eastmoney_topic.csv",usecols=['dominant_topic'])

In [18]:
data['topic'] = topic_data['dominant_topic']

In [19]:
import pyreadr

# Load the RData file
result = pyreadr.read_r('./data/industry_csrc2012_matched.RData')
ind_data = result['industry_csrc2012_matched']
ind_data[['stock_code','IndustryCode1','IndustryCode2']]


,stock_code,IndustryCode1,IndustryCode2
0,000001,J,J66
1,000002,K,K70
2,000004,C,C27
3,000004,I,I65
4,000005,N,N77
...,...,...,...
5405,600270,G,G58
5406,600401,C,C39
5407,600432,C,C32
5408,600747,F,F51


In [20]:
ind_data[['stock_code','IndustryCode1','IndustryCode2']]
ind_data['stock_code'] = ind_data['stock_code'].astype(int)
ind_data = ind_data.drop_duplicates(subset=['stock_code'])

In [21]:
merged_data = data.merge(ind_data[['stock_code','IndustryCode1','IndustryCode2']], on='stock_code', how='left')
len(merged_data)

6603696

In [23]:
print(merged_data['IndustryCode1'].isna().sum())
merged_data['IndustryCode1'].fillna('others', inplace=True)
print(merged_data['IndustryCode1'].isna().sum())

0
0


In [24]:
print(merged_data['IndustryCode2'].isna().sum())
merged_data['IndustryCode2'].fillna('others', inplace=True)
print(merged_data['IndustryCode2'].isna().sum())

13250
0


C:\Users\ZIQING002\AppData\Local\Temp\ipykernel_31388\4104984895.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged_data['IndustryCode2'].fillna('others', inplace=True)


In [25]:
data = merged_data
data.columns

Index(['item_title', 'article_author', 'year', 'month',
       'eastmoney_robo_journalism', 'media_robo_journalism',
       'SMA_robo_journalism', 'viral', 'article_source_cate',
       'item_author_cate', 'stock_code', 'sentiment', 'topic', 'IndustryCode1',
       'IndustryCode2'],
      dtype='object')

In [26]:
data['year'] = data['year'].astype(int)
data['month'] = data['month'].astype(int)
data['eastmoney_robo_journalism'] = data['eastmoney_robo_journalism'].astype(int)
data['media_robo_journalism'] = data['media_robo_journalism'].astype(int)
data['SMA_robo_journalism'] = data['SMA_robo_journalism'].astype(int)
data['stock_code'] = data['stock_code'].astype(int)
data['topic'] = data['topic'].astype(int)
data['sentiment'] = data['sentiment'].map({'Neutral': 1, 'Positive': 2, 'Negative': 0}).astype(int)
data['viral'] = data['viral'].astype(int)
data['item_author'] = data['item_author_cate'].astype(str)
data['article_author'] = data['article_author'].astype(str)
data['article_source'] = data['article_source_cate'].astype(str)
data.loc[data['article_source'].str.len() > 40, 'article_source'] = "东方财富Choice数据"
data.loc[data['article_source']=='东方财富Choice数据■', 'article_source'] = "东方财富Choice数据"

In [27]:
data.describe(include='all')

,item_title,article_author,year,month,eastmoney_robo_journalism,media_robo_journalism,SMA_robo_journalism,viral,article_source_cate,item_author_cate,stock_code,sentiment,topic,IndustryCode1,IndustryCode2,item_author,article_source
count,6603696,6603696,6.603696e+06,6.603696e+06,6.603696e+06,6.603696e+06,6.603696e+06,6.603696e+06,6603696,6603696,6.603696e+06,6.603696e+06,6.603696e+06,6603696,6603696,6603696,6603696
unique,6405069,5799,NaN,NaN,NaN,NaN,NaN,NaN,219,5158,NaN,NaN,NaN,20,82,5158,219
top,百联股份今日热门盘点,0,NaN,NaN,NaN,NaN,NaN,NaN,东方财富,股市胖虎,NaN,NaN,NaN,C,C39,股市胖虎,东方财富
freq,116,3253327,NaN,NaN,NaN,NaN,NaN,NaN,4181680,426876,NaN,NaN,NaN,4241203,765036,426876,4181680
mean,NaN,NaN,2.021472e+03,6.593989e+00,5.504324e-01,1.353547e-01,2.612287e-01,4.778476e-02,NaN,NaN,3.414867e+05,1.081507e+00,2.177143e+00,NaN,NaN,NaN,NaN
std,NaN,NaN,1.164505e+00,3.552674e+00,4.974501e-01,3.421020e-01,4.393043e-01,2.133105e-01,NaN,NaN,2.724346e+05,4.752776e-01,1.072451e+00,NaN,NaN,NaN,NaN
min,NaN,NaN,2.017000e+03,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NaN,NaN,1.000000e+00,0.000000e+00,0.000000e+00,NaN,NaN,NaN,NaN
25%,NaN,NaN,2.021000e+03,4.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NaN,NaN,2.522000e+03,1.000000e+00,2.000000e+00,NaN,NaN,NaN,NaN
50%,NaN,NaN,2.022000e+03,7.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NaN,NaN,3.008900e+05,1.000000e+00,2.000000e+00,NaN,NaN,NaN,NaN
75%,NaN,NaN,2.022000e+03,1.000000e+01,1.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,NaN,NaN,6.010000e+05,1.000000e+00,3.000000e+00,NaN,NaN,NaN,NaN


index categorical features

In [28]:
data['item_author_index'], uniques = pd.factorize(data['item_author'])

In [29]:
data['article_author_index'], uniques = pd.factorize(data['article_author'])

In [30]:
data['article_source_index'], uniques = pd.factorize(data['article_source'])

In [31]:
data['stock_code_index'], uniques = pd.factorize(data['stock_code'])

In [32]:
data['ind_code1_index'], uniques = pd.factorize(data['IndustryCode1'])

In [33]:
data['ind_code2_index'], uniques = pd.factorize(data['IndustryCode2'])

In [34]:
data.to_csv('./data/eastmoney_full.csv', index=False)

## Rank author info based on past month's stats

In [35]:
import pandas as pd
data = pd.read_csv('./data/eastmoney_full.csv')

C:\Users\ZIQING002\AppData\Local\Temp\ipykernel_31388\2751206451.py:2: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('./data/eastmoney_full.csv')


In [36]:
data.columns

Index(['item_title', 'article_author', 'year', 'month',
       'eastmoney_robo_journalism', 'media_robo_journalism',
       'SMA_robo_journalism', 'viral', 'article_source_cate',
       'item_author_cate', 'stock_code', 'sentiment', 'topic', 'IndustryCode1',
       'IndustryCode2', 'item_author', 'article_source', 'item_author_index',
       'article_author_index', 'article_source_index', 'stock_code_index',
       'ind_code1_index', 'ind_code2_index'],
      dtype='object')

In [37]:
# Create a 'year_month' column for easier handling
data['year_month'] = pd.to_datetime(data['year'].astype(str) + '-' + data['month'].astype(str) + '-01')

In [38]:
def rank_by_prev_month(data, col):
    # Group by 'year', 'month', and col to be ranked to count the number of posts by each author in each month
    monthly_count = data.groupby(['year', 'month', col]).size().reset_index(name='post_count')

    # Create a shifted version of 'author_monthly_posts' to simulate the "previous month"
    monthly_count['year_month'] = pd.to_datetime(monthly_count['year'].astype(str) + '-' + monthly_count['month'].astype(str) + '-01')
    monthly_count['previous_year_month'] = monthly_count['year_month'] - pd.DateOffset(months=1)

    # Split 'previous_year_month' into year and month to prepare for the merge
    monthly_count['previous_year'] = monthly_count['previous_year_month'].dt.year
    monthly_count['previous_month'] = monthly_count['previous_year_month'].dt.month

    # Split 'previous_year_month' into year and month to prepare for the merge
    monthly_count['previous_year'] = monthly_count['previous_year_month'].dt.year
    monthly_count['previous_month'] = monthly_count['previous_year_month'].dt.month

    # Rank authors by their post count within the previous month
    monthly_count[f"{col}_rank"] = monthly_count.groupby(['previous_year', 'previous_month'])['post_count'].rank(ascending=False, method='dense')

    # Merge the rank information back into the original data by matching on 'year', 'month', and 'item_author_index'
    data_with_rank = pd.merge(data, 
                            monthly_count[[col, 'year', 'month', f"{col}_rank"]], 
                            how='left', 
                            left_on=[col, 'year', 'month'], 
                            right_on=[col, 'year', 'month'])

    # Display the updated data with the author rank in the previous month
    print(data_with_rank[['year', 'month', col, f"{col}_rank"]])

    return data_with_rank

In [39]:
def reduce_dim_by_rank(df, col, max_rank=10):
    # Create a mask for ranks that are greater than max_rank
    rank_mask = df[f"{col}_rank"] > max_rank

    # Set ranks exceeding max_rank to max_rank + 1
    df.loc[rank_mask, f"{col}_rank"] = max_rank + 1
    
    # Set corresponding column values (e.g., 'item' or 'author') to max_index + 1
    # df.loc[rank_mask, col] = max_index + 1

    return df

In [40]:
data = rank_by_prev_month(data, 'item_author_index')

         year  month  item_author_index  item_author_index_rank
0        2020      5                  0                     3.0
1        2020      8                  1                     2.0
2        2020      9                  2                     1.0
3        2020      9                  2                     1.0
4        2020      9                  2                     1.0
...       ...    ...                ...                     ...
6603691  2023      8               5111                    55.0
6603692  2023      8               5111                    55.0
6603693  2023      8               5111                    55.0
6603694  2023      8               5111                    55.0
6603695  2023      8               5111                    55.0

[6603696 rows x 4 columns]


In [41]:
# data = data.apply(reduce_dim_by_rank, axis=1, col='item_author_index', max_index=data['item_author_index'].max())
data = reduce_dim_by_rank(data, 'item_author_index', max_rank=10)
print(data[['year', 'month', 'item_author_index', "item_author_index_rank"]])

         year  month  item_author_index  item_author_index_rank
0        2020      5                  0                     3.0
1        2020      8                  1                     2.0
2        2020      9                  2                     1.0
3        2020      9                  2                     1.0
4        2020      9                  2                     1.0
...       ...    ...                ...                     ...
6603691  2023      8               5111                    11.0
6603692  2023      8               5111                    11.0
6603693  2023      8               5111                    11.0
6603694  2023      8               5111                    11.0
6603695  2023      8               5111                    11.0

[6603696 rows x 4 columns]


In [42]:
data = rank_by_prev_month(data, 'article_author_index')

         year  month  article_author_index  article_author_index_rank
0        2020      5                     0                        1.0
1        2020      8                     0                        1.0
2        2020      9                     0                        1.0
3        2020      9                     0                        1.0
4        2020      9                     0                        1.0
...       ...    ...                   ...                        ...
6603691  2023      8                     0                        2.0
6603692  2023      8                     0                        2.0
6603693  2023      8                   452                       48.0
6603694  2023      8                     0                        2.0
6603695  2023      8                     0                        2.0

[6603696 rows x 4 columns]


In [43]:
data = reduce_dim_by_rank(data, 'article_author_index', max_rank=10)
print(data[['year', 'month', 'article_author_index', "article_author_index_rank"]])

         year  month  article_author_index  article_author_index_rank
0        2020      5                     0                        1.0
1        2020      8                     0                        1.0
2        2020      9                     0                        1.0
3        2020      9                     0                        1.0
4        2020      9                     0                        1.0
...       ...    ...                   ...                        ...
6603691  2023      8                     0                        2.0
6603692  2023      8                     0                        2.0
6603693  2023      8                   452                       11.0
6603694  2023      8                     0                        2.0
6603695  2023      8                     0                        2.0

[6603696 rows x 4 columns]


In [44]:
data = rank_by_prev_month(data, 'article_source_index')

         year  month  article_source_index  article_source_index_rank
0        2020      5                     0                        1.0
1        2020      8                     0                        1.0
2        2020      9                     1                        1.0
3        2020      9                     1                        1.0
4        2020      9                     1                        1.0
...       ...    ...                   ...                        ...
6603691  2023      8                    30                       14.0
6603692  2023      8                    57                       20.0
6603693  2023      8                     6                       15.0
6603694  2023      8                    21                       21.0
6603695  2023      8                    15                        8.0

[6603696 rows x 4 columns]


In [45]:
data = reduce_dim_by_rank(data, 'article_source_index', max_rank=10)
print(data[['year', 'month', 'article_source_index', "article_source_index_rank"]])

         year  month  article_source_index  article_source_index_rank
0        2020      5                     0                        1.0
1        2020      8                     0                        1.0
2        2020      9                     1                        1.0
3        2020      9                     1                        1.0
4        2020      9                     1                        1.0
...       ...    ...                   ...                        ...
6603691  2023      8                    30                       11.0
6603692  2023      8                    57                       11.0
6603693  2023      8                     6                       11.0
6603694  2023      8                    21                       11.0
6603695  2023      8                    15                        8.0

[6603696 rows x 4 columns]


In [46]:
data = data[['item_title', 'item_author', 'article_author',
       'article_source', 'year', 'month', 'eastmoney_robo_journalism',
       'media_robo_journalism', 'SMA_robo_journalism', 'stock_code', 'viral',
       'sentiment', 'topic', 'item_author_index', 'article_author_index',
       'article_source_index', 'stock_code_index', 'year_month', 'ind_code1_index', 'ind_code2_index',
       'item_author_index_rank', 'article_author_index_rank', 'article_source_index_rank']]

In [47]:
data.to_csv('./data/eastmoney_full_ranked.csv', index=False)

save metadata: unique value count for each feature

In [48]:
len(data)

6603696

In [49]:
post_ft = ['month', 
            'ind_code1_index',
            'ind_code2_index',
            'sentiment',
            'topic']
author_ft = ['eastmoney_robo_journalism',
                'media_robo_journalism',
                'SMA_robo_journalism',
                'item_author_index_rank',
                'article_author_index_rank',
                'article_source_index_rank']

In [50]:
post_ft_unique_count = [int(data[x].max()+1) for x in post_ft]
author_ft_unique_count = [int(data[x].max()+1) for x in author_ft]

In [51]:
post_ft_unique_count, author_ft_unique_count

([13, 20, 82, 3, 5], [2, 2, 2, 12, 12, 12])

In [52]:
import pickle
meta_data = (post_ft_unique_count, author_ft_unique_count)
with open('./data/bpr_v3_meta.pkl','wb') as f:
    pickle.dump(meta_data, f)

# Sample negative BPR samples

Split the data choronologically

In [53]:
import pandas as pd
data = pd.read_csv('./data/eastmoney_full_ranked.csv', usecols=['item_title', 'item_author', 'article_author',
       'article_source', 'year', 'month', 'eastmoney_robo_journalism',
       'media_robo_journalism', 'SMA_robo_journalism', 'stock_code', 'viral',
       'sentiment', 'topic', 'item_author_index', 'article_author_index',
       'article_source_index', 'stock_code_index', 'year_month', 'ind_code1_index', 'ind_code2_index',
       'item_author_index_rank', 'article_author_index_rank', 'article_source_index_rank'])

C:\Users\ZIQING002\AppData\Local\Temp\ipykernel_31388\195616194.py:2: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('./data/eastmoney_full_ranked.csv', usecols=['item_title', 'item_author', 'article_author',


First round:
- Train 2017-01-01, 2021.12.13 --> train_bpr1.csv
- Valid 2022-01-01, 2022-06-30 --> valid_bpr.csv
- Test  2022-07-01, 2022-09-30 --> test1.csv

Second round: 
- Train 2022-07-01, 2022-09-30 --> train_bpr2.csv (test1)
- Test  2022-10-01, 2022-12-31 --> test2.csv

Third round:
- Train 2022-10-01, 2022-12-31 --> train_bpr3.csv (test2)
- Test  2023-01-01, 2022-03-31 --> test3.csv

Fourth round:
- Train 2023-01-01, 2022-03-31 --> train_bpr4.csv
- Test  2023-04-01, 2023-08-31 --> test4.csv

In [54]:
## first round 

# Define the date ranges for train, valid, and test sets
train1_start = '2017-01-01'
train1_end = '2021-12-31'
valid1_start = '2022-01-01'
valid1_end = '2022-06-30'
test1_start = '2022-07-01'
test1_end = '2022-09-30'

test2_start = '2022-10-01'
test2_end = '2022-12-31'

test3_start = '2023-01-01'
test3_end = '2023-03-31'

test4_start = '2023-04-01'
test4_end = '2023-08-31'

# Split the data based on the defined ranges
train_set = data[(data['year_month'] >= train1_start) & (data['year_month'] <= train1_end)]
valid_set = data[(data['year_month'] >= valid1_start) & (data['year_month'] <= valid1_end)]
test1_set = data[(data['year_month'] >= test1_start) & (data['year_month'] <= test1_end)]
test2_set = data[(data['year_month'] >= test2_start) & (data['year_month'] <= test2_end)]
test3_set = data[(data['year_month'] >= test3_start) & (data['year_month'] <= test3_end)]
test4_set = data[(data['year_month'] >= test4_start) & (data['year_month'] <= test4_end)]

In [55]:
len(train_set), len(valid_set), len(test1_set), len(test2_set), len(test3_set), len(test4_set)

(3208461, 1112397, 451944, 423289, 426113, 981492)

In [56]:
len(train_set[train_set['viral']==1]), len(valid_set[valid_set['viral']==1]), len(test1_set[test1_set['viral']==1]), len(test2_set[test2_set['viral']==1]), len(test3_set[test3_set['viral']==1]), len(test4_set[test4_set['viral']==1])

(259955, 9436, 4506, 9963, 11906, 19790)

In [57]:
train_set.to_csv("./data/train1.csv", index=False)
valid_set.to_csv("./data/valid1.csv", index=False)
test1_set.to_csv("./data/test1.csv", index=False)
test2_set.to_csv("./data/test2.csv", index=False)
test3_set.to_csv("./data/test3.csv", index=False)
test4_set.to_csv("./data/test4.csv", index=False)

sample bpr data

In [58]:
train_set = pd.read_csv("./data/train1.csv")
valid_set = pd.read_csv("./data/valid1.csv")
test1_set = pd.read_csv("./data/test1.csv")
test2_set = pd.read_csv("./data/test2.csv")
test3_set = pd.read_csv("./data/test3.csv")
test4_set = pd.read_csv("./data/test4.csv")

C:\Users\ZIQING002\AppData\Local\Temp\ipykernel_31388\2589231190.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  train_set = pd.read_csv("./data/train1.csv")
C:\Users\ZIQING002\AppData\Local\Temp\ipykernel_31388\2589231190.py:2: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  valid_set = pd.read_csv("./data/valid1.csv")
C:\Users\ZIQING002\AppData\Local\Temp\ipykernel_31388\2589231190.py:3: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  test1_set = pd.read_csv("./data/test1.csv")
C:\Users\ZIQING002\AppData\Local\Temp\ipykernel_31388\2589231190.py:4: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  test2_set = pd.read_csv("./data/test2.csv")
C:\Users\ZIQING002\AppData\Local\Temp\ipykernel_31388\2589231190.py:5: DtypeWarning: Columns (2) have mixed types. Specify dtype o

In [59]:
from tqdm import tqdm
def sample_bpr(data, aux_data=[]):

    #combine data
    ref_data = pd.concat([data] + aux_data, axis=0, ignore_index=True)

    # Separate positive and negative samples
    positive_samples = data[data['viral'] == 1]
    negative_samples = ref_data[ref_data['viral'] == 0]

    # Create dictionaries to group negative samples by criteria for faster lookup
    negatives_by_criteria1 = negative_samples.groupby(['item_author', 'article_author', 'article_source'])
    negatives_by_criteria21 = negative_samples.groupby('article_source')
    negatives_by_criteria22 = negative_samples.groupby('stock_code')

    # List to hold the result rows
    sampled_negatives = []

    # Loop over each positive sample
    for _, positive_row in tqdm(positive_samples.iterrows(), total=len(positive_samples)):
        # Try to get negative samples by Criteria 1
        key1 = (positive_row['item_author'], positive_row['article_author'], positive_row['article_source'])
        matching_negatives_criteria1 = negatives_by_criteria1.get_group(key1) if key1 in negatives_by_criteria1.groups else pd.DataFrame()

        # If Criteria 1 matches fewer than 3 rows, add Criteria 2 matches
        if len(matching_negatives_criteria1) < 3:
            key2 = positive_row['article_source']
            matching_negatives_criteria21 = negatives_by_criteria21.get_group(key2) if key2 in negatives_by_criteria21.groups else pd.DataFrame()
            matching_negatives_criteria21 = matching_negatives_criteria21.sample(n=3, replace=False) if len(matching_negatives_criteria21) >= 3 else matching_negatives_criteria21
            key3 = positive_row['stock_code']
            matching_negatives_criteria22 = negatives_by_criteria22.get_group(key3) if key3 in negatives_by_criteria22.groups else pd.DataFrame()
            matching_negatives_criteria22 = matching_negatives_criteria22.sample(n=3, replace=False) if len(matching_negatives_criteria22) >= 3 else matching_negatives_criteria22
            combined_matches = pd.concat([matching_negatives_criteria1, matching_negatives_criteria21, matching_negatives_criteria22]).drop_duplicates()
            sampled = combined_matches.sample(n=3, replace=False) if len(combined_matches) >= 3 else combined_matches
        else:
            # Criteria 1 provides enough matches
            sampled = matching_negatives_criteria1.sample(n=3, replace=False)
        
        if len(sampled)<3:
            print(key1, key2, key3)
            print(negatives_by_criteria22.groups.keys())
            return key3, negatives_by_criteria22
        
        # Prefix columns and append each pair to result list
        for _, negative_row in sampled.iterrows():
            combined_row = pd.concat([positive_row, negative_row.add_prefix('neg_')])
            sampled_negatives.append(combined_row)

    # Concatenate all combined rows into a final DataFrame
    final_sampled_negatives = pd.DataFrame(sampled_negatives).reset_index(drop=True)

    return final_sampled_negatives

In [94]:
# round1
train_bpr1 = sample_bpr(train_set)

100%|██████████| 259955/259955 [23:37<00:00, 183.43it/s] 


In [47]:
train_bpr1.to_csv('./data/train_bpr1.csv', index=False)
print(len(train_set), len(train_set[train_set['viral']==1]), len(train_bpr1))

3208461 259955 779865


In [48]:
valid_bpr = sample_bpr(valid_set, [train_set])
valid_bpr.to_csv('./data/valid_bpr.csv', index=False)
print(len(valid_set), len(valid_set[valid_set['viral']==1]), len(valid_bpr))

100%|██████████| 9436/9436 [00:37<00:00, 249.01it/s]


1112397 9436 28308


In [49]:
# round2
train_bpr2 = sample_bpr(test1_set, [train_set, valid_set])
train_bpr2.to_csv('./data/train_bpr2.csv', index=False)
print(len(test1_set), len(test1_set[test1_set['viral']==1]), len(train_bpr2))

100%|██████████| 4506/4506 [00:29<00:00, 153.35it/s]


451944 4506 13518


In [50]:
# round3
train_bpr3 = sample_bpr(test2_set, [train_set, valid_set, test1_set])
train_bpr3.to_csv('./data/train_bpr3.csv', index=False)
print(len(test2_set), len(test2_set[test2_set['viral']==1]), len(train_bpr3))

100%|██████████| 9963/9963 [00:50<00:00, 198.21it/s]


423289 9963 29889


In [51]:
# round4
train_bpr4 = sample_bpr(test3_set, [train_set, valid_set, test1_set])
train_bpr4.to_csv('./data/train_bpr4.csv', index=False)
print(len(test3_set), len(test3_set[test3_set['viral']==1]), len(train_bpr4))

100%|██████████| 11906/11906 [01:19<00:00, 149.35it/s]


426113 11906 35718


Appendix: bpr sample analysis

In [112]:
import pandas as pd
# check avg matching statistics
raw_data = pd.read_csv('./data/eastmoney_bert.csv', usecols=['viral','item_author_cate', 'article_author', 'article_source_cate', 'stock_code', 'year', 'month'])

C:\Users\ZIQING002\AppData\Local\Temp\ipykernel_26456\2214158559.py:3: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_data = pd.read_csv('./data/eastmoney_bert.csv', usecols=['viral','item_author_cate', 'article_author', 'article_source_cate', 'stock_code', 'year', 'month'])


In [113]:
raw_data['item_author'] = raw_data['item_author_cate'].astype(str)
raw_data['article_author'] = raw_data['article_author'].astype(str)
raw_data['article_source'] = raw_data['article_source_cate'].astype(str)
raw_data.loc[raw_data['article_source'].str.len() > 40, 'article_source'] = "东方财富Choice数据"
raw_data.loc[raw_data['article_source']=='东方财富Choice数据■', 'article_source'] = "东方财富Choice数据"

In [114]:
raw_data['year_month'] = pd.to_datetime(raw_data['year'].astype(int).astype(str) + '-' + raw_data['month'].astype(int).astype(str) + '-01')
raw_train = raw_data[(raw_data['year_month'] >= '2017-01-01') & (raw_data['year_month'] <= '2021-12-31')]
print(len(raw_train))

3208461


In [118]:
positive_rows = raw_train[raw_train['viral'] == 1]
negative_rows = raw_train[raw_train['viral'] == 0]


# C1
neg_count_df = negative_rows.groupby(['item_author', 'article_author', 'article_source']).size().reset_index(name='neg_count1')
# Merge with unique combinations to get primary counts
merged = positive_rows.merge(neg_count_df, on=['item_author', 'article_author', 'article_source'], how='left',suffixes=('', '1')).fillna(0)
print(merged.head())

# C2
neg_count_df21 = negative_rows.groupby(['item_author']).size().reset_index(name='neg_count21')
neg_count_df22 = negative_rows.groupby(['article_author']).size().reset_index(name='neg_count22')
neg_count_df23 = negative_rows.groupby(['article_source']).size().reset_index(name='neg_count23')

merged = merged.merge(neg_count_df21, on='item_author', how='left', suffixes=('', '21')).fillna(0)
merged = merged.merge(neg_count_df22, on='article_author', how='left', suffixes=('', '22')).fillna(0)
merged = merged.merge(neg_count_df23, on='article_source', how='left', suffixes=('', '23')).fillna(0)
merged['neg_count2'] = merged.apply(lambda row: row['neg_count21'] + row['neg_count22'] + row['neg_count23'], axis=1)

# C3
neg_count_df3 = negative_rows.groupby(['stock_code']).size().reset_index(name='neg_count3')
# Merge with the previous result on stock_code
merged = merged.merge(neg_count_df3, on='stock_code', how='left', suffixes=('', '3')).fillna(0)

# Adjust neg_count2 based on whether neg_count was found
# merged['neg_count2'] = merged.apply(lambda row: row['neg_count2'] if row['neg_count'] == 0 else 0, axis=1)
merged['total'] = merged.apply(lambda row: row['neg_count1'] + row['neg_count2'] + row['neg_count3'], axis=1)

# Convert to lists if needed
neg_count1 = merged['neg_count1'].tolist()
neg_count21 = merged['neg_count21'].tolist()
neg_count22 = merged['neg_count22'].tolist()
neg_count23 = merged['neg_count23'].tolist()
neg_count2 = merged['neg_count2'].tolist()
neg_count3 = merged['neg_count3'].tolist()
neg_count_t = merged['total'].tolist()

  article_author    year  month  viral article_source_cate item_author_cate  \
0              0  2021.0    5.0    1.0                东方财富            胖猫周周见   
1              0  2021.0    5.0    1.0                东方财富            大眼看两融   
2              0  2021.0    5.0    1.0                东方财富            胖猫周周见   
3              0  2021.0    5.0    1.0                东方财富            大眼看两融   
4              0  2021.0    5.0    1.0            Choice数据     股友8635377Z3T   

   stock_code   item_author article_source year_month  neg_count1  
0           1         胖猫周周见           东方财富 2021-05-01     44390.0  
1           1         大眼看两融           东方财富 2021-05-01     72999.0  
2           1         胖猫周周见           东方财富 2021-05-01     44390.0  
3           1         大眼看两融           东方财富 2021-05-01     72999.0  
4           1  股友8635377Z3T       Choice数据 2021-05-01       311.0  


In [14]:
import numpy as np
def analyse_neg_sample(neg_count1, neg_count2=None):
    if neg_count2:
        neg_count = np.array([sum(x) for x in zip(neg_count1, neg_count2)])
    else:
        neg_count = np.array(neg_count1)

    # Mean of the array
    mean = np.mean(neg_count)

    # Median of the array
    median = np.median(neg_count)

    # Count of zeros in the array
    zero_count = np.size(neg_count) - np.count_nonzero(neg_count)

    print(f"Mean: {mean}")
    print(f"Median: {median}")
    print(f"Zero count: {zero_count}")

In [15]:
analyse_neg_sample(neg_count1)

Mean: 609.5727251467198
Median: 3.0
Zero count: 63527


In [16]:
analyse_neg_sample(neg_count1, neg_count21)

Mean: 2391.9262633124345
Median: 937.0
Zero count: 535


In [17]:
analyse_neg_sample(neg_count1, neg_count22)

Mean: 1906228.6984754503
Median: 2221475.0
Zero count: 2945


In [18]:
analyse_neg_sample(neg_count1, neg_count23)

Mean: 181237.7250539179
Median: 21698.0
Zero count: 12


In [19]:
analyse_neg_sample(neg_count2)

Mean: 2088029.6316172404
Median: 2243655.0
Zero count: 0


In [21]:
analyse_neg_sample(neg_count3)

Mean: 1259.1293361693597
Median: 1320.0
Zero count: 530


In [22]:
analyse_neg_sample(neg_count1, neg_count3)

Mean: 1868.7020613160796
Median: 1348.0
Zero count: 530


In [23]:
analyse_neg_sample(neg_count21, neg_count3)

Mean: 3041.4828743350745
Median: 2248.0
Zero count: 530


In [24]:
analyse_neg_sample(neg_count22, neg_count3)

Mean: 1906878.2550864727
Median: 2222497.0
Zero count: 2


In [25]:
analyse_neg_sample(neg_count23, neg_count3)

Mean: 181887.28166494056
Median: 22521.0
Zero count: 0
